In [1]:
!pip install konlpy

     |████████████████████████████████| 19.4MB 1.3MB/s 
     |████████████████████████████████| 92kB 7.5MB/s 
     |████████████████████████████████| 3.6MB 38.9MB/s 
  Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3
  Found existing installation: tweepy 3.6.0
    Uninstalling tweepy-3.6.0:
      Successfully uninstalled tweepy-3.6.0


## 4.1.2. Process of Preprocessing

### 1. collecting corpus

- crawling : `robots.txt` 통해 crawling 가능 여부 확인


In [2]:
!wget https://www.mobileindex.com/robots.txt

--2020-06-01 03:46:42--  https://www.mobileindex.com/robots.txt
Resolving www.mobileindex.com (www.mobileindex.com)... 114.207.244.51
Connecting to www.mobileindex.com (www.mobileindex.com)|114.207.244.51|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 57 [text/plain]
Saving to: ‘robots.txt’

robots.txt          100%[===================>]      57  --.-KB/s    in 0s      

2020-06-01 03:46:43 (7.80 MB/s) - ‘robots.txt’ saved [57/57]



In [6]:
!cat robots.txt

﻿User-agent: *
Disallow: /member/
Disallow: /console/

In [3]:
!wget https://www.ted.com/robots.txt

--2020-06-01 03:47:44--  https://www.ted.com/robots.txt
Resolving www.ted.com (www.ted.com)... 151.101.2.49, 151.101.66.49, 151.101.130.49, ...
Connecting to www.ted.com (www.ted.com)|151.101.2.49|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 394 [text/plain]
Saving to: ‘robots.txt.1’

robots.txt.1        100%[===================>]     394  --.-KB/s    in 0s      

2020-06-01 03:47:44 (8.56 MB/s) - ‘robots.txt.1’ saved [394/394]



In [5]:
!cat robots.txt.1

User-agent: *
Disallow: /latest
Disallow: /latest-talk
Disallow: /latest-playlist
Disallow: /people
Disallow: /profiles
Disallow: /conversations
Disallow: /themes/rss
Disallow: /discussions
Disallow: /tpv4

User-agent: Baiduspider
Disallow: /search
Disallow: /latest
Disallow: /latest-talk
Disallow: /latest-playlist
Disallow: /people
Disallow: /profiles
Disallow: /discussions
Disallow: /tpv4


### 2. normalization ; 정제

- 전각 문자 제거
- 대소문자 통일 ; 임베딩에 따라 필요성 줄어드는 추세
- 정규 표현식 사용한 정제 ; regexper.com 참고, 치환자 사용



In [16]:
import re

x = '''abcdefg
12345
ab12
a1bc2d
12ab
a1b
1a2
a1
1a
hijklmnop'''

regex = r'([a-z])[0-9]+([a-z])'
to = r'\1\1\2'

y = '\n'.join([re.sub(regex, to, x_i) for x_i in x.split('\n')])
print (y)

abcdefg
12345
ab12
aabccd
12ab
aab
1a2
a1
1a
hijklmnop


### 3. 문장 단위 분절

- 대부분의 경우, 한 라인에 한 문장만 있어야 함
- 직접 분절 module / algorithm 만들기 보다는 NLP toolkit NLTK 등 사용 후 추가 전/후처리하는 것을 권장




한 라인에 여러 문장 있는 경우 예제

In [0]:
import sys, fileinput, re
from nltk.tokenize import sent_tokenize

In [0]:
if __name__ == '__main__' :
    for line in fileinput.input(files = ('/content/robots.txt.1')) :
        if line.strip() != "":
            line = re.sub(r'([a-z])\.([A-Z])', r'\1. \2', line.strip() )
            sentences = sent_tokenize(line.strip())

            for s in sentences :
                if s!= "": sys.stdout.write(s + '\n')

여러 라인에 한 문장 들어있는 경우

In [0]:
if __name__ == '__main__' : 
    buf = []
    for line in fileinput.input():
        if line.strip() != '':
            buf += [line.strip()]
            sentences = sent_tokenize(" ".join(buf))

            if len(sentences) > 1 :
                buf = sentences[-1:]
                sys.stdout.write(sentences[0] + '\n')

    sys.stdout.write(' '.join(buf) + '\n')


### 4. 분절

- 형태소 분석 또는 단순 분절


#### 한국어 분절

- Mecab : 가장 많이 사용, C++로 제작
- KoNLPy


In [26]:
!bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)

Installing automake (A dependency for mecab-ko)
Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:6 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:7 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:9 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:12 http://ppa.launchpad.net/marutter/c2d4u3.5/ubuntu bionic InRelease [15.4 kB]
Get:13 http://security.ubuntu.com/ubuntu bionic-security/universe amd64 Packages [854 kB]
Get:14 http://archive

In [27]:
!echo '안녕하세요, 반갑습니다!' | mecab

안녕	NNG,행위,T,안녕,*,*,*,*
하	XSV,*,F,하,*,*,*,*
세요	EP+EF,*,F,세요,Inflect,EP,EF,시/EP/*+어요/EF/*
,	SC,*,*,*,*,*,*,*
반갑	VA,*,T,반갑,*,*,*,*
습니다	EF,*,F,습니다,*,*,*,*
!	SF,*,*,*,*,*,*,*
EOS


In [28]:
!echo '안녕하세요, 반갑습니다!' | mecab -O wakati

안녕 하 세요 , 반갑 습니다 ! 


#### 영어분절

- Moses ; NLTK 3.2.5 버전까지 포함

In [29]:
!pip install nltk==3.2.5

In [0]:
from nltk.tokenize.moses import MosesTokenizer

t = MosesTokenizer()

input_lines = u"Natural language processing is one of biggest stream in artificial intelligence, and it becomes very popular after seq2seq's invention. However, in order to make a strong A.I., there are still many challenges remain. I believe that we can breakthrough these barriers to get strong artificial intelligence."

for line in input_lines : 
    if line.strip() != '':
        tokens = t.tokenize(line.strip(), escape = False)
        sys.stdout.write(' '.join(tokens) + '\n')
    else : sys.stdout.write('\n')

- 중국어 분절 : Stanford Lex-Parser, JIEBA


### 5. 병렬 corpus 정렬 (optional)


### 6. sub-word 분절

- BPE algorithm : byte pair encoding, 단어는 의미를 가진 더 작은 subword들의 조합
- 필수 전처리 방법
- 희소성 감소, UNKnown-token에 대한 효율적 대처
- Google SentencePiece module : https://github.com/google/sentencepiece
- 논문 원저자 Sennrich의 코드를 저자가 수정한 버전 : https://github.com/kh-kim/subword-nmt

### 7. detokenization ; 분절 복원

- 분절에 따른 특수문자 제거


In [0]:
STR = '▁'

if __name__ == "__main__":
    ref_fn = sys.argv[1]

    f = open(ref_fn, 'r')

    for ref in f:
        ref_tokens = ref.strip().split(' ')
        input_line = sys.stdin.readline().strip()

        if input_line != "":
            tokens = input_line.split(' ')

            idx = 0
            buf = []

            # We assume that stdin has more tokens than reference input.
            for ref_token in ref_tokens:
                tmp_buf = []

                while idx < len(tokens):
                    if tokens[idx].strip() == '':
                        idx += 1
                        continue

                    tmp_buf += [tokens[idx]]
                    idx += 1

                    if ''.join(tmp_buf) == ref_token:
                        break

                if len(tmp_buf) > 0:
                    buf += [STR + tmp_buf[0].strip()] + tmp_buf[1:]

            sys.stdout.write(' '.join(buf) + '\n')
        else:
            sys.stdout.write('\n')

    f.close()

In [0]:
if __name__ == "__main__":
    for line in sys.stdin:
        if line.strip() != "":
            if '▁▁' in line:
                line = line.strip().replace(' ', '').replace('▁▁', ' ').replace('▁', '').strip()
            else:
                line = line.strip().replace(' ', '').replace('▁', ' ').strip()

            sys.stdout.write(line + '\n')
        else:
            sys.stdout.write('\n')

### TorchText

- preprocess library for NLP and ML/DL for Text

In [39]:
!pip install torchtext

Read corpus and label

- tsv format
- 주로 텍스트 분류에 사용

In [0]:
from torchtext import data

class DataLoader(object):
    def __init__(self, train_fn, valid_fn, batch_size = 64, device = -1, max_vocab = 999999, min_freq = 1, use_eos = False, shuffle = True) :
        super().__init__()

        self.label = data.Field(sequential = False, use_vocab = True, unk_token = None)
        self.text = data.Field(use_vocab = True, batch_first = True, include_lengths = False, eos_token = '<EOS>' if use_eos else None)

        train, valid = data.TabularDataset.splits(
            path = '',
            train = train_fn,
            validation = valid_fn,
            format = 'tsv',
            fileds = [
                      ('label', self.label),
                      ('text', self.text),
            ]
        )
        self.train_iter, self.valid_iter = data.BucketIterator.splits((train, valid), 
                                                                      batch_size = batch_size,
                                                                      device = f'cuda : {device}' if device >= 0 else 'cpu',
                                                                      shuffle = shuffle,
                                                                      sort_key = lambda x : len(x.text),
                                                                      sort_within_batch = True
                                                                      )
        self.label.build_vocab(train)
        self.text.build_vocab(train, max_size = max_vocab, min_freq = min_freq)




Read Corpus

- 한 라인이 텍스트로만 채워져 있을 때
- 주로 언어 모델 훈련에 사용

In [0]:
from torchtext import data, datasets

In [0]:
PAD, BOS, EOS = 1,2,3

class DataLoader():

    def __init__(self, train_fn, valid_fn, batch_size = 64, device = 'cpu', max_vocab = 99999999999, max_length = 255, fix_length = None, use_bos = True, use_eos = True, shuffle = True) :
        super().__init__()

        self.text = data.Field(
            sequential = True,
            use_vocab = True,
            batch_first = True,
            include_lengths = True,
            fix_length = fix_length,
            init_token = '<BOS>' if use_bos else None,
            eos_token = '<EOS>' if use_eos else None
            )

        train = LanguageModelDataset (
            path = train_fn,
            fields = [('text', self.text)],
            max_length = max_length
        )
        valid = LanguageModelDataset(
            path = valid_fn,
            fields = [('text', self.text)],
            max_length = max_length
        )

        self.train_iter = data.BucketIterator(
            dataset = train, 
            batch_size = batch_size,
            device = f"cuda : {device}" if device >= 0 else 'cpu', 
            shuffle = shuffle, 
            sort_key = lambda x : -len(x.text), 
            sort_within_batch = True
        )
        self.valid_iter = data.BucketIterator(
            dataset = valid,
            batch_size = batch_size,
            shuffle = False, 
            sort_key = lambda x : -len(x.text), 
            sort_within_batch = True
        )
        self.text.build_vocab(train, max_size = max_vocab)

class LanguageModelDataset(data.Dataset) :
    def __init__(self, path, fields, max_length = None, **kwargs) : 
        if not isinstance(fields[0], (tuple, list)) :
            fields = [('text', fields[0])]
        examples = []
        with open(path) as f :
            for line in f  :
                line = line.strip()
                if max_length and max_length < len(line.split()) :
                    continue
                if line != '' :
                    examples.append(data.Example.fromlist( [line], fields ))
        super().__init__(examples, fields, **kwargs)
                

Read parallel corpus data

- 주로 기계번역, 요약 등
- 같은 라인끼리 mapping 되어야 하므로, 같은 수로 채워져야 함

In [0]:
import os
from torchtext import data, datasets

PAD, BOS, EOS = 1,2,3

class DataLoader() :
    def __init__(
        self,
        train_fn = None,
        valid_fn = None,
        exts = None,
        batch_size = 64,
        device = 'cpu',
        max_vocab = 9999999,
        max_length = 255,
        fix_length = None,
        use_bos = True,
        use_eos = True,
        shuffle = True,
        dsl = False
    ) :
        super().__init__()
        self.src = data.Field(
            sequential = True, 
            use_vocab = True, 
            batch_first = True, 
            include_lengths = True, 
            fix_length = fix_length, 
            init_token = '<BOS>' if dsl else None, 
            eos_token = '<EOS>'  if dsl else None
            # dtype=torch.dtype instance, preprocessing=None, postprocessing=None, lower=False, tokenize=<function Field.<lambda> at 0x7f00019a81e0>, pad_token='<pad>', unk_token='<unk>', pad_first=False, truncate_first=False, stop_words=None, is_target=False
        )

        self.tgt = data.Field(
            sequential = True, 
            use_vocab = True, 
            batch_first = True, 
            include_lengths = True, 
            fix_length = fix_length, 
            init_token = '<BOS>' if dsl else None, 
            eos_token = '<EOS>'  if dsl else None
            # dtype=torch.dtype instance, preprocessing=None, postprocessing=None, lower=False, tokenize=<function Field.<lambda> at 0x7f00019a81e0>, pad_token='<pad>', unk_token='<unk>', pad_first=False, truncate_first=False, stop_words=None, is_target=False
        )

        if train_fn is not None and valid_fn is not None and exts is not None :
            train = TranslationDataset (
                path = train_fn,
                exts = exts,
                fields = [
                          ('src', self.src),
                          ('tgt', self.tgt)
                ],
                max_length = max_length
            )
            valid = TranslationDataset (
                path = valid_fn,
                exts = exts,
                fields = [
                          ('src', self.src),
                          ('tgt', self.tgt)
                ]
                max_length = max_length
            )
        
        self.train_iter = data.BucketIterator(
            dataset = train, 
            batch_size = batch_size, 
            device = f"cuda {device}" if device >= 0 else 'cpu', 
            shuffle = shuffle, 
            sort_key = lambda x : len(x.tgt) + (max_length * len(x.src)), 
            sort_within_batch = True
            # batch_size_fn=None, train=True, repeat=False, sort=None, 
        )
        self.valid_iter = data.BucketIterator(
            dataset = valid, 
            batch_size = batch_size, 
            device = f"cuda {device}" if device >= 0 else 'cpu', 
            shuffle = False, 
            sort_key = lambda x : len(x.tgt) + (max_length * len(x.src)), 
            sort_within_batch = True
            # batch_size_fn=None, train=True, repeat=False, sort=None, 
        )

        self.src.build_vocab(train, max_size = max_vocab)
        self.tgt.build_vocab(train, max_size = max_vocab)

    def load_vocab(self, src_vocab, tgt_vocab) :
        self.src.vocab = src_vocab
        self.tgt.vocab = tgt_vocab

class TranslationDataset(data.Dataset) :
    
    @staticmethod
    def sort_key(ex) :
        return data.interleave_keys(len(ex.src), len(ex.trg))
    
    def __init__(self, path, exts, fields, max_length = None, **kwargs) :
        if not isinstance(fields[0], (tuple, list)) :
            fields = [('src', fields[0]), ('trg', fields[1])]
        if not path.endswith('.') : 
            path += '.'
        
        src_path, trg_path = tuple(os.path.expanduser(path + x) for x in exts)
        
        examples = []
        with open(src_path, encoding='UTF-8') as src_file, open(trg_path, encoding='UTF-8') as trg_file :
            for src_line, trg_line in zip (src_file, trg_file) :
                src_line, trg_line = src_line.strip(), trg_line.strip()
                if max_length and max_length < max(len(src_line.split()), len(trg_line.split())) : continue
                if src_line != '' and trg_line != '' :
                    examples.append(data.Example.fromlist([src_line, trg_line], fields = fields))
        
        super().__init__(examples, fields, **kwargs)

In [0]:
data.Field??

In [0]:
isinstance?

In [0]:
**kwargs?